## Exploratory data analysis

### Attribution

These notes borrow heavily from the book "Think Stats", by Allen B. Downey.

### Approach

Let's start with a demographical question: do first babies tend to arrive late?

This simple question is not easy to answer from anecdotal evidence, and it is a good problem to start building our modeling toolkit with Python.

This toolkit includes techniques in data collection, descriptive statistics, exploratory data analysis, estimation, and hypothesis testing.

### Data source

We will use the National Survey of Family Growth (NSFG), which is stored in a `.csv` file stored in this repository. In order to use it, we will use a library called `pandas`.

`pandas` is one of the most useful Python packages for data analysis and modeling. Let's use it to read our input data:

In [53]:
import pandas as pd
df = pd.read_csv('.lesson/assets/FemPreg.csv')

In [54]:
df

,row_number,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13588,13588,12571,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,6.1875
13589,13589,12571,2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,NaN
13590,13590,12571,3,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,NaN
13591,13591,12571,4,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,7.5000


This dataset has been prepared to be easy to read, with columns that have (relatively) descriptive names:

In [55]:
df.columns

Index(['row_number', 'caseid', 'pregordr', 'howpreg_n', 'howpreg_p',
       'moscurrp', 'nowprgdk', 'pregend1', 'pregend2', 'nbrnaliv',
       ...
       'laborfor_i', 'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt',
       'finalwgt', 'secu_p', 'sest', 'cmintvw', 'totalwgt_lb'],
      dtype='object', length=245)

Each column in the dataframe can be extracted and treated as a pandas' `Series`, which can be thought of as an improved list. Let's examine the "pregnancy serial number".

In [56]:
pregord = df['pregordr']
type(pregord)
pregord

0        1
1        2
2        1
3        2
4        3
        ..
13588    1
13589    2
13590    3
13591    4
13592    5
Name: pregordr, Length: 13593, dtype: int64

We can see the components of a Series: the indices, the elements, the variable name, the length, and the data type.

### Variables

The most important variables in the dataset are the following:

- caseid is the integer ID of the respondent

- prglngth is the integer duration of the pregnancy in weeks.

- outcome is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth.

- pregordr is a pregnancy serial number; for example, the code for a respondent’s first pregnancy is 1, for the second pregnancy is 2, and so on.

- birthord is a serial number for live births; the code for a respondent’s first child is 1, and so on. For outcomes other than live birth, this field is blank.

- birthwgt_lb and birthwgt_oz contain the pounds and ounces parts of the birth weight of the baby.

- agepreg is the mother’s age at the end of the pregnancy.

- finalwgt is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.

Some of the variables are `recodes`, which means that they are calculated from the `raw data`. For instance you could do:

In [57]:
df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0

if it was not already calculated.

Note that you may use the dot notation to access attributes but not to add new columns to a dataframe (if you try, you will be adding an attribute that will not be treated as a column, and this will generate confusion).

### Validation

One way to validate data is to compute basic statistics. In our case, we may want to look into the encoding of each pregnancy and count the number of times each value appears by using the `value_counts()` command.

In [58]:
df.outcome.value_counts().sort_index()

1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: outcome, dtype: int64

The encoding for this variable is as follows:

| value | label |
|----|----|
| 1 | LIVE BIRTH |
| 2 | INDUCED ABORTION |
| 3 | STILLBIRTH |
| 4 | MISCARRIAGE |
| 5 | ECTOPIC PREGNANCY |
| 6 | CURRENT PREGNANCY |

We can do something similar with the weigth of the newborn babies:

In [59]:
df.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

Imagine that we are told that the baby weighing 15 pounds is actually a typo, and that the weight of that baby is actually unknown. Pandas provides an easy way to deal with the situation:

In [60]:
import numpy as np
df.loc[df.birthwgt_lb == 15.0, 'birthwgt_lb'] = np.nan
df.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
Name: birthwgt_lb, dtype: int64

The `.loc` attribute provides several ways to select rows and columns from a dataframe. The first expresion in brackets is the row indexer, whereas the second selects the column.

### Interpretation

Let's transform our data so that we see how many pregnancies each particular respodent had in a more explicit way.

We will use `defaultdict`, which is a `container` from the package `collections`. A container is just a constructor of a data structure, in this case a dictionary. `defaultdict` has the advantage over `dict` that it allows to create items that you try to access if they do not exist. You may think of it as a tweaked version of the regular `dict`.

In [61]:
from pprint import pprint
import collections
d1 = collections.defaultdict(list)
for index, caseid in df.caseid.items():
    d1[caseid].append(index)

There is a particular case (i.e. a particular pregnant woman) worth our attention: `10229`. Let's see in which rows (i.e. in which pregnancy records) she appears and what was the outcome:

In [62]:
d1[10229]
df.outcome[d1[10229]]

11093    4
11094    4
11095    4
11096    4
11097    4
11098    4
11099    1
Name: outcome, dtype: int64

If we go back to our encoding table we will see that this is a remarkable case in which the woman had six consecutive miscarriages and, finally, a live birth.

## Distributions

### Histograms
One of the best ways to understand a variable is to understand which values it takes and to count of many times each occurs. This is called the **distribution** of the variable.

The most common representation of a distribution is a **histogram**, which is a plot that shows the **frequency** of each value.

An histogram can be calculated with a dictionary. Say `t` is a series of values:

In [63]:
t = (0,1,1,2,2,2,1,1,1,2,2,1,2,1,0,1,0,0,1,1,2,1,2,0,1,2,0,1,2,1,2,1,2,)
hist = {}
for x in t:
    hist[x] = hist.get(x,0) + 1 

### Representing histograms

The most popular plotting library in Python is `matplotlib`. Other libraries are built on top of it to provide an easier syntax and ready-to-use plots; `seaborn` and `plotly` are in this camps and are the ones we will use the most.

These libraries require some configuration, so you may want to include the following commands in your code:

In [74]:
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
sns.set_style("darkgrid")
mpl.rcParams["axes.labelsize"] = 14
mpl.rcParams["xtick.labelsize"] = 12
mpl.rcParams["ytick.labelsize"] = 12
mpl.rcParams["text.color"] = "k"
mpl.rcParams["figure.dpi"] = 200

We also need first a directory to store our plots:

In [75]:
from pathlib import Path
Path('plots').mkdir(parents=True, exist_ok=True)

For Mac users working locally with VSCode, it may be necessary to get into the applications folder and execute a `Install certificates.command` file.

Let's get back to the plot itself:

In [76]:
import pandas as pd
df = pd.read_csv('.lesson/assets/FemPreg.csv')
hist0 = sns.histplot(data=df, x="birthwgt_lb", discrete=True)
fig = hist0.get_figure()
fig.savefig('plots/birthwgt_lb.png')
plt.clf()

<Figure size 1280x960 with 0 Axes>

(`discrete` is an aesthetics parameter that avoids "gaps" between bins in the case of discrete variables such as this).

The distribution is close to normal, with some assymetry due to a tail that extends farther to the left than to the right.

We can plot other variables in the dataset and we will see that they have different distributions. Whereas the `birthwgt_lb` is close to a normal (i.e. Gaussian), the `birthwgt_oz` is bounded and close to uniform, with the exception of the zero value, probably due to rounding.

In [77]:
hist1 = sns.histplot(data=df, x="birthwgt_oz", discrete=True)
fig = hist1.get_figure()
fig.savefig('plots/birthwgt_oz.png')
plt.clf()

<Figure size 1280x960 with 0 Axes>

The age of pregnancy is also bell-shaped, but in this case the right tail is longer than the left tail.

In [78]:
hist2 = sns.histplot(data=df, x="agepreg", discrete=True)
fig = hist2.get_figure()
fig.savefig('plots/agepreg.png')
plt.clf()

<Figure size 1280x960 with 0 Axes>

Finally, the length of pregnancy in weeks has a longer left tail, because early babies are more common than pregnancies that go past 43 weeks.

In [79]:
hist3 = sns.histplot(data=df, x="prglngth", discrete=True)
fig = hist3.get_figure()
fig.savefig('plots/prglngth.png')
plt.clf()

<Figure size 1280x960 with 0 Axes>

### Outliers

Checking for outliers (i.e. extreme values) is a good practice, since they may be either measurement errors or actual rare events, and they typically need to be taken care of.

In [80]:
(df.
    query('outcome == 1').
    nsmallest(10, 'prglngth').
    loc[:,'prglngth']
)

6458      0
4108      4
138       9
11887    13
8919     17
10492    17
541      18
7762     19
1041     20
7800     21
Name: prglngth, dtype: int64

This is a technique that we will use often: concatenating different operations to a dataframe; in this case a filter with `query`, a extraction of the lowest values with `nsmallest`, and a column selection with `loc`. Note the (optional) enclosing parenthesis for a cleaner syntax.

On the other end of the range we have:

In [81]:
(df.
    query('outcome == 1').
    nlargest(10, 'prglngth').
    loc[:,'prglngth']
)

4783     50
8998     50
2416     48
4920     48
6889     48
6890     48
7158     48
7161     48
7592     48
11357    47
Name: prglngth, dtype: int64

Is it really possible that a pregnancy lasts 50 weeks? The best way to discern whether this datapoint is an error or an actual extreme value is through **domain knowledge**.

### First babies

Now we can take a look at the question "are first babies born earlier?" by adding an additional column to the dataset and plotting two histograms:

In [82]:
transform = lambda x: True if x==1 else False
df = df.assign(firstborn = df.birthord.map(transform))

Now we can plot the two groups. However, the histograms are not "normalized" and therefore it is not immediate to compare the two distributions. The way to address the problem is through probability mass functions.

In [83]:
hist4 = sns.histplot(data=df, x="prglngth", discrete=True, hue='firstborn')
fig = hist4.get_figure()
fig.savefig('plots/first_born.png')
plt.clf()

<Figure size 1280x960 with 0 Axes>

### Summarize distributions

A histogram is a complete description of the distribution of a sample. Often we want to summarize the distribution with a few descriptive statistics:
- central tendency: do the values tend to cluster around a particular point?
- modes: is there more than one cluster?
- spread: how much variability is there in the values?
- tails: how quickly do the probabilities drop off as we move away from the modes?
- outliers: are there extreme values far from the modes?
Statistics designed to answer these questions are called summary statistics. 

By far the most common summary statistic is the mean, which is meant to describe the central tendency of the distribution and is calculated as follows:

![The mean](.lesson/assets/mean.png)

The variance, which is the squared power of the standard deviation, is another summary statistic intended to describe the variability of a distribution.


![The variance](.lesson/assets/variance.png)

Pandas provides methods to compute mean, variance and standard deviation:

In [84]:
df.query('outcome == 1').prglngth.mean()
df.query('outcome == 1').prglngth.var()
df.query('outcome == 1').prglngth.std()

2.702343810070587

So, for all live births, the mean pregnancy length is 38.56 weeks, and the standard deviation is 2.7 weeks, meaning that deviation of 2-3 weeks are common.